- Author: Addison Li 
- Date: 2019-03-17
- Content: JPM filtering with curve fitting on Ticker, Sector, and S&P Rating 

In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit

pd.set_option('display.max_columns', 27)

In [222]:
def clean_names(name_list):
    col_names = []
    for name in name_list:
        t = name.lower()
        if " " in t:
            c_t = t.replace(' ',"_")
        else:
            c_t = t
        col_names.append(c_t)
    return col_names

In [223]:
def fill_time(df1):
    
    copy = df1.iloc[:,0].apply(lambda x: None if len(x) <= 12 else x)
    copy = copy.ffill()
    
    return copy
    
 

In [224]:
''' 
The first step of preparing the dataframe: 
1. drop the na columns or rows after examnination 
2. clean up column names 


def prep_df_1 (raw_df):
    temp = raw_df.dropna(axis = 1, how = 'all')
    names = temp.columns.tolist()
    col_names = clean_names(names)
    
    temp.columns = col_names
    
    # not the most ideal, but this way keeps the original idx in tact
    temp_2 = temp.dropna(axis = 0, thresh = 16)
    
    return temp_2
   

def prep_df_2(raw):
    df1 =  prep_df_1(raw) #(58324, 20)
    dates = fill_time(df1)
    df1.insert(0, column = 'dates', value = pd.Series(dates))
    df1 = df1.dropna() # (58308, 22)
    df2 = df1.reset_index()
    df3 = df2.drop(columns = ['index', 'unnamed:_0'], axis = 1)
    
    return df3
 
def clean_time(p2_df):
    from datetime import datetime
    
    start_date,end_date = p2_df['dates'].str.split('-', 1).str
    p2_df.insert(0,'start_date', start_date)
    p2_df.insert(1, 'end_date', end_date)
    
    p2_df.start_date = p2_df.start_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
    p2_df.end_date = p2_df.end_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
    
    p2_df.drop(labels = ['dates'], axis = 1, inplace = True)
    
    return p2_df

clean_df = clean_time(p2_df)

'''
  

" \nThe first step of preparing the dataframe: \n1. drop the na columns or rows after examnination \n2. clean up column names \n\n\ndef prep_df_1 (raw_df):\n    temp = raw_df.dropna(axis = 1, how = 'all')\n    names = temp.columns.tolist()\n    col_names = clean_names(names)\n    \n    temp.columns = col_names\n    \n    # not the most ideal, but this way keeps the original idx in tact\n    temp_2 = temp.dropna(axis = 0, thresh = 16)\n    \n    return temp_2\n   \n\ndef prep_df_2(raw):\n    df1 =  prep_df_1(raw) #(58324, 20)\n    dates = fill_time(df1)\n    df1.insert(0, column = 'dates', value = pd.Series(dates))\n    df1 = df1.dropna() # (58308, 22)\n    df2 = df1.reset_index()\n    df3 = df2.drop(columns = ['index', 'unnamed:_0'], axis = 1)\n    \n    return df3\n \ndef clean_time(p2_df):\n    from datetime import datetime\n    \n    start_date,end_date = p2_df['dates'].str.split('-', 1).str\n    p2_df.insert(0,'start_date', start_date)\n    p2_df.insert(1, 'end_date', end_date)\n  

In [225]:
raw = pd.read_csv('vanguard_merge.csv', thousands = ',', parse_dates = ['Mty','Iss Dt','date'])

# raw.shape: (1541005, 27)


In [227]:
col_names = raw.columns

In [228]:
clean_col = clean_names(col_names)

In [229]:
raw.columns = clean_col

In [230]:
raw.isna().sum()

bclass3                     0
country                     0
bid_spread                  0
cur_yld                     0
g_spd                       0
years_to_mat                0
oas                         0
oad                         0
amt_out                     0
cpn                         0
excess_rtn                  0
isin                        0
ticker                      0
mty                       268
iss_dt                      0
px_close                    0
krd_6m                      0
krd_2y                      0
krd_5y                      0
krd_10y                     0
krd_20y                     0
krd_30y                     0
s&p_rating_num           9986
accrued_int_(%)             0
yield_to_mat                0
class_-_detail_-_code     105
date                        0
dtype: int64

In [231]:
# dropping missing data because there is 1 bond, missing 268 days of mty,
# same for s&p rating, 9986 out of 1,541,005 is 0.6%. we can safely toss them out. 
df = raw.dropna()
# shape: (1530646, 27)

In [251]:
df[df.date == "2019-01-23"].tail(3)

,bclass3,country,bid_spread,cur_yld,g_spd,years_to_mat,oas,oad,amt_out,cpn,excess_rtn,isin,ticker,mty,iss_dt,px_close,krd_6m,krd_2y,krd_5y,krd_10y,krd_20y,krd_30y,s&p_rating_num,accrued_int_(%),yield_to_mat,class_-_detail_-_code,date
1359936,Consumer Non-Cyclical,United States,93.0,3.27,91.0,2.57,91.00,2.41,300000.0,3.250,0.04,US98978VAQ68,ZTS,2021-08-20,2018-08-20,99.40,0.00,1.87,0.53,0.0,0.0,0.0,10.0,1.390,3.50,BBDE,2019-01-23
1359937,Banking,Netherlands,73.0,1.94,73.0,2.49,72.85,2.39,300000.0,1.875,0.00,XS1451467127,RABOBK,2021-07-19,2016-07-19,96.55,-0.01,1.92,0.47,0.0,0.0,0.0,6.0,0.964,3.31,BAA,2019-01-23
1359938,Banking,S.Korea,169.0,3.92,168.0,2.87,168.39,2.67,500000.0,3.875,0.07,XS1523140942,SHNHAN,2026-12-07,2016-12-07,98.95,0.00,1.86,0.80,0.0,0.0,0.0,9.0,0.506,4.27,BAA,2019-01-23


In [11]:
# vectorized version of calculating g_spd 

def mark_gspd_v(clean_df, today_date, isin_list):
    
    target_df = clean_df[clean_df['isin'].isin(isin_list)]

    out_df = target_df[target_df.date == today_date][['isin', 'g_spd']]
    out_df['g_min'] = target_df.groupby(['isin'])['g_spd'].min().values
    out_df['g_max'] = target_df.groupby(['isin'])['g_spd'].max().values
    out_df['delta'] = out_df.g_max - out_df.g_min
   
    # filter out the bonds with delta great than 10 base points 

    delta_df = out_df[out_df.delta >= 10] 
    c_filter = delta_df['g_spd'] > delta_df['g_min'] + delta_df['delta'].values*0.95
    delta_df['cheap']= [1 if i == True else 0 for i in c_filter]

    r_filter = delta_df['g_spd'] < delta_df['g_min'] + delta_df['delta'].values*0.05 
    delta_df['rich']= [1 if i == True else 0 for i in r_filter]

    return delta_df 

In [253]:
# looking at the last month:  2019-02-07  to 2019-03-07

one_month = df.iloc[1359938:, :]

In [254]:
len(one_month.date.unique())


30

In [255]:
today_date = '2019-03-07'
today_df = one_month[one_month.date == today_date]
isin_all = today_df['isin']


In [256]:
# delta_df returns a dataframe that's filtered by 2 criterias:
# 1) we are only look at the ones delta > 10 over the target period (30 / 60 days)
# 2) g_spd exceeds top 95% or falls below the bottom 5%
delta_df = mark_gspd_v(one_month, today_date, isin_all)

/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [257]:
# expensive bonds  
expensive = delta_df[delta_df['rich'] == 1]

print("There are", expensive.shape[0], "expensive bonds.")

There are 1050 expensive bonds.


In [258]:
# cheap bonds - good
cheap = delta_df[delta_df['cheap'] == 1]

print("There are", cheap.shape[0], "cheap bonds.")

There are 197 cheap bonds.


### look at individual bonds spread over 10 day time period. If this bond spread is trading at its widest(cheapest) 5% level, over the course of 10 day, mark this criteria 


### Fit the issuer curve
- compare the spread against the issuer curve, which is just all the bonds within the same issuer and if that individual bond is trading wide to the issuer curve it meets another criteria
- issuer curve, y-axis: g_spread , x-axis: years_to_ maturity 

In [259]:
ticker_groupby = one_month.groupby(['ticker', 'date','isin'])['g_spd', 'years_to_mat'].mean() # shape:(58314, 2)

In [260]:
np.random.seed(1729)

def func(x, m, c):
    return m*np.log(x)+ c 

def cal_curve(ticker_groupby):
    
    from scipy.optimize import curve_fit

    tickers = pd.unique(ticker_groupby.index.get_level_values(0))
    dates = pd.unique(ticker_groupby.index.get_level_values(1))
    ticker_temp = ticker_groupby.reset_index()
    
    g_spd_pred =[]
    diff = []
    idx = []
    
    for t in tickers:
        for d in dates:
            temp = ticker_temp[ticker_temp['ticker'] == t]
            sub_df = temp[temp['date'] == d]
            
            if sub_df.shape[0] >= 6:
                sub_x = sub_df['years_to_mat'].values
                sub_y = sub_df['g_spd'].values
            
                popt, pcov = curve_fit(func, sub_x, sub_y)
                g_predict = func(sub_x, popt[0], popt[1])
                sub_diff = g_predict - sub_y
                g_spd_pred.append(g_predict)
                diff.append(sub_diff)
                idx.append(sub_df.index)
            
    g_pred = np.concatenate(g_spd_pred)
    g_diff = np.concatenate(diff)
    g_idx = np.concatenate(idx)
    
    return g_pred, g_diff, g_idx 
    
    

In [261]:
g_pred_t, g_diff_t, g_idx_t = cal_curve(ticker_groupby)  

In [262]:
result_ticker = ticker_groupby.iloc[g_idx_t]

In [263]:
result_ticker['g_pred'] = g_pred_t

/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [264]:
result_ticker['g_diff'] = g_diff_t  # result_df.shape : (54806, 4)

/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [265]:
peer_df = result_ticker.reset_index() # peer_df.shape: (54806, 7), unique isin 5572


In [266]:
peer_df.shape

(140203, 7)

- the total num of unique isins in the dataset is 5925 
- after filtering the ones we cant fit the line with peers, there are 5572 unique isin in peer_df
- the num of target date unique isin is 5832
- the intersect between target date isin and peer isin is 5481
- however, when we get all the peer isin info on the target date, we are missing 4 isins


In [267]:
def self_peer_filter(peer_df, target_isin, target_date):
    
    avail_isin = np.intersect1d(target_isin, peer_df['isin'].values) 
    
    prep_self = peer_df[peer_df['isin'].isin(avail_isin)]
    self_df = prep_self.groupby(['isin'])[['g_diff']].max().reset_index()
    self_df.columns = ['isin', 'g_diff_max']
    
    self_df['g_diff_min'] = prep_self.groupby(['isin'])[['g_diff']].min().values
    self_df['delta'] = self_df['g_diff_max'] - self_df['g_diff_min']
    self_df['perc_95'] = self_df.g_diff_min + self_df.delta.values*0.95
    self_df['bottom_5'] = self_df.g_diff_min + self_df.delta.values*0.05
   
    # making sure delta is >= 10
    delta_10 = self_df[self_df['delta']>=10]
    delta_isin = delta_10['isin'].values
    sub_peer = peer_df[peer_df['isin'].isin(delta_isin)]
    
    target_diff = sub_peer[sub_peer['date'] == target_date][['isin', 'g_diff']]  #1429 rows × 7 columns
    sub_self = self_df[self_df['isin'].isin(target_diff['isin'].values)]
    
    return target_diff, sub_self

In [268]:
def get_results(target_diff, sub_self):
    c = target_diff['g_diff'].values - sub_self['perc_95'].values
    cheap =[1 if diff >0 else 0 for diff in c] 
    sub_self['cheap'] = cheap
    
    r = target_diff['g_diff'].values - sub_self['bottom_5'].values
    rich = [1 if diff < 0 else 0 for diff in r]
    sub_self['rich']= rich
    
    sub_self['target_day'] = target_diff['g_diff'].values
    
    return sub_self

In [270]:
target_date = "2019-03-07"
target_isin = one_month[one_month['date'] == target_date]['isin'].values
#len(target_isin)

In [271]:
target_diff, sub_self = self_peer_filter(peer_df, target_isin, target_date)

In [272]:
ticker_final = get_results(target_diff, sub_self)

In [273]:
ticker_final.head(10)

,isin,g_diff_max,g_diff_min,delta,perc_95,bottom_5,cheap,rich,target_day
0,US001055AJ19,9.742960,-1.871478,11.614438,9.162238,-1.290756,0,0,0.099140
1,US001055AL64,14.656741,0.554859,14.101882,13.951647,1.259953,0,1,-1.782996
2,US001055AM48,17.761705,-0.628311,18.390016,16.842204,0.291190,1,0,26.010219
7,US00108WAD20,14.746628,0.342575,14.404053,14.026425,1.062777,0,1,-13.939129
9,US00108WAH34,6.722973,-6.687988,13.410961,6.052425,-6.017440,0,0,-5.382505
15,US001192AJ21,13.223852,-14.715790,27.939643,11.826870,-13.318808,0,0,-6.692355
16,US001192AK93,2.660700,-8.444020,11.104719,2.105464,-7.888784,0,0,-4.071720
21,US00185AAF12,23.268738,7.121001,16.147737,22.461351,7.928387,0,1,5.768969
22,US00185AAG94,30.539427,20.457131,10.082296,30.035312,20.961246,0,1,-11.451835
23,US00185AAH77,32.325171,21.928792,10.396380,31.805352,22.448611,0,1,4.069176


In [274]:
result_cheap = one_month_result_df[one_month_result_df['cheap']==1]
print("There are",result_cheap.shape[0], "bonds are cheap." )


There are 816 bonds are cheap.


In [275]:
result_rich = one_month_result_df[one_month_result_df['rich']==1]
print("There are",result_rich.shape[0], "bonds are rich." )

There are 838 bonds are rich.


### Further filter by subsector / bclass3 + S&P rating


In [278]:
one_month.bclass3.unique()

array(['Insurance', 'Banking', 'Capital Goods', 'Electric',
       'Consumer Cyclical', 'Basic Industry', 'Energy',
       'Consumer Non-Cyclical', 'Technology', 'Other Industrial', 'REITs',
       'Communications', 'Brokerage Assetmanagers Exchanges',
       'Natural Gas', 'Transportation', 'Other Financial',
       'Finance Companies', 'Other Utility'], dtype=object)

In [279]:
sector_groupby = one_month.groupby(['bclass3','s&p_rating_num', 'date','isin'])['g_spd', 'years_to_mat'].mean() 

In [280]:
pd.unique(sector_groupby.index.get_level_values(1))


array([ 5.,  6.,  7.,  8.,  9., 10., 11., 13.,  4., 12.,  2.,  3., 21.,
       23.])

In [281]:
np.random.seed(1729)

def func(x, m, c):
    return m*np.log(x)+ c 

def cal_curve(sector_groupby):
    
    from scipy.optimize import curve_fit

    sectors = pd.unique(sector_groupby.index.get_level_values(0))
    ratings = pd.unique(sector_groupby.index.get_level_values(1))
    dates = pd.unique(sector_groupby.index.get_level_values(2))
    sector_temp = sector_groupby.reset_index()
    
    g_spd_pred =[]
    diff = []
    idx = []
    
    for s in sectors:
        for r in ratings:
            for d in dates:
                temp = sector_temp[sector_temp['bclass3'] == s]
                temp2 = temp[temp['s&p_rating_num'] == r]
                sub_df = temp2[temp2['date'] == d]
            
                if sub_df.shape[0] >= 6:
                    sub_x = sub_df['years_to_mat'].values
                    sub_y = sub_df['g_spd'].values
            
                    popt, pcov = curve_fit(func, sub_x, sub_y)
                    g_predict = func(sub_x, popt[0], popt[1])
                    sub_diff = g_predict - sub_y
                    g_spd_pred.append(g_predict)
                    diff.append(sub_diff)
                    idx.append(sub_df.index)
            
        g_pred = np.concatenate(g_spd_pred)
        g_diff = np.concatenate(diff)
        g_idx = np.concatenate(idx)
    
    return g_pred, g_diff, g_idx 
    
    

In [282]:
def self_peer_filter(peer_df, target_isin, target_date):
    
    avail_isin = np.intersect1d(target_isin, peer_df['isin'].values) 
    
    prep_self = peer_df[peer_df['isin'].isin(avail_isin)]
    self_df = prep_self.groupby(['isin'])[['g_diff']].max().reset_index()
    self_df.columns = ['isin', 'g_diff_max']
    
    self_df['g_diff_min'] = prep_self.groupby(['isin'])[['g_diff']].min().values
    self_df['delta'] = self_df['g_diff_max'] - self_df['g_diff_min']
    self_df['perc_95'] = self_df.g_diff_min + self_df.delta.values*0.95
    self_df['bottom_5'] = self_df.g_diff_min + self_df.delta.values*0.05
   
    # making sure delta is >= 10
    delta_10 = self_df[self_df['delta']>=10]
    delta_isin = delta_10['isin'].values
    sub_peer = peer_df[peer_df['isin'].isin(delta_isin)]
    
    target_diff = sub_peer[sub_peer['date'] == target_date][['isin', 'g_diff']]  #1429 rows × 7 columns
    sub_self = self_df[self_df['isin'].isin(target_diff['isin'].values)]
    
    return target_diff, sub_self

In [283]:
def get_results(target_diff, sub_self):
    c = target_diff['g_diff'].values - sub_self['perc_95'].values
    cheap =[1 if diff >0 else 0 for diff in c] 
    sub_self['cheap'] = cheap
    
    r = target_diff['g_diff'].values - sub_self['bottom_5'].values
    rich = [1 if diff < 0 else 0 for diff in r]
    sub_self['rich']= rich
    
    sub_self['target_day'] = target_diff['g_diff'].values
    
    return sub_self

In [284]:
g_pred_s, g_diff_s, g_idx_s = cal_curve(sector_groupby)  

In [285]:
result_sector = sector_groupby.iloc[g_idx_s]

In [286]:
result_sector['g_pred'] = g_pred_s

/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [287]:
result_sector['g_diff'] = g_diff_s

/Users/AddisonYing/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [289]:
peer_sector = result_sector.reset_index() 
peer_sector.shape

(169379, 8)

In [290]:

target_isin = one_month[one_month['date'] == target_date]['isin'].values 

target_diff_s, sub_self_s = self_peer_filter(peer_sector, target_isin, target_date)
    

In [291]:
sector_final = get_results(target_diff_s, sub_self_s)

In [292]:
sector_final.shape

(4286, 9)

In [293]:
sector_final.rich.sum()

1744

In [294]:
sector_final.cheap.sum()

1707

In [295]:
sector_isin_r = list(sector_final[sector_final['rich']==1]['isin'])


In [296]:
sector_isin_c = list(sector_final[sector_final['cheap']==1]['isin'])

In [297]:
ticker_isin_r = list(ticker_final[ticker_final['rich'] ==1]['isin'])

In [298]:
ticker_isin_c = list(ticker_final[ticker_final['cheap'] ==1]['isin'])

In [299]:
cheap_final = np.intersect1d(sector_isin_c, ticker_isin_c)

In [301]:
rich_final = np.intersect1d(sector_isin_r, ticker_isin_r)

In [300]:
len(cheap_final)

471

In [302]:
len(rich_final)

442

### Merging Elin's results with Addy's

In [315]:
elin = pd.read_csv('elin.csv')


In [304]:
elin_rich = elin['Rich bonds'].dropna()

In [312]:
elin_cheap = elin['Cheap bonds'].dropna()

In [320]:
elin_addy_rich = np.intersect1d(elin_rich, rich_final)
print( "There are", len(elin_addy_rich), "rich bonds of the target date:", target_date)

There are 47 rich bonds of the target date: 2019-03-07


In [322]:
elin_addy_cheap = np.intersect1d(elin_cheap, cheap_final)
print( "There are", len(elin_addy_cheap), "cheap bonds of the target date:", target_date)

There are 50 cheap bonds of the target date: 2019-03-07
